In [1]:
# nbi:hide_in
# nbi:hide_out
%pylab inline
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:80% !important; }</style>'));
display(HTML('<style>.text_cell { width:80% !important; font-family: Verdana, Geneva, sans-serif }</style>'));

Populating the interactive namespace from numpy and matplotlib


# Covid-19 in Schools
<font color="#1E8449"><i>Jenny Hoffman, updated Aug 18, 2020</i></font>

I'm trying to figure out whether to send my kids back to school this fall, so I built a simulator to help parents, teachers, and administrators understand the risks. The total probability that a person will become infected at school is the product of:  

$\quad$ (probability that an infected person comes to school) $\times$ (probability that the virus is transmitted)

You can look up the first probability using this
<a href="https://covid19risk.biosci.gatech.edu/" target="_blank">Risk Assessment Tool</a>.
The school can influence the second probability with the following policies:

- Strictly enforce mask wearing ($F_{\mathrm{mask}}$ = mask filtration % effectiveness)
- Increase the fresh air flow ($R_{\mathrm{fresh}}$ = $F_{\mathrm{fresh}}\,R_{\mathrm{hvac}}$ = fresh air flow in ft$^3$/min)
- Use at least one
<a href="https://www.amazon.com/gp/product/B08194ZQ4N/" target="_blank">HEPA filter</a>
per classroom ($R_{\mathrm{hepa}}$ = filtration rate in ft$^3$/min; $F_{\mathrm{hepa}}$ = filtration %)
- Spend as little time indoors as possible ($t$ = hours spent indoors per week)
- Don't move students between classrooms; move teachers instead if necessary.

Note that physical distance (the oft-touted "6ft separation") does not prevent virus transmission indoors, because aerosols diffuse quickly throughout a room at ambient temperature, and building HVAC systems can carry the virus over large distances in hard-to-predict directions. For more detail about how 9 people were sickened across multiple tables in a restaurant and 92 people were sickened across a large floor of a workspace, see <a href="https://www.erinbromage.com/post/the-risks-know-them-avoid-them" target="_blank">Erin Bromage's post</a>.

### Simulation for Parents

In [2]:
# nbi:hide_in

# initialize parameters
Vroom = 20*30*9;  # volume of room in cubic feet
airturnovers = 4; # number of air turnovers per hour

Rhvac = Vroom*airturnovers/60;    # cubic feet per minute of air pushed into the room by HVAC system
Ffresh = 10;      # % of hvac air that is fresh (building code requires 10% minimum fresh outside air)
Rfan = 0;         # cubic feet per minute of fresh air pushed in by window fan

Rleak = 0;        # leakage of hallway air into the room
nhall = 0;        # number of virus particles per cubic foot in hallway air

Nhepa = 1;        # number of free-standing hepa filters in the room
Rhepa = 240;      # cubic feet per minute processed by one free-standing hepa filter
Fhepa = 99.9;     # % of virus particles removed by HEPA filter

Nsick = 1;        # number of sick people in the room
p = 33;           # number of virus particles shed per minute by 1 sick person
m = 25;           # mask filtration efficiency (0% if no mask, 100% if inpenetrable mask)

k = 410;          # number of inhaled virus particles required to cause infection
Lperhour = 450;   # healthy adult L per hour breathed
ft3perL = 0.0353147;  # ft^3 per liter
Vbreath = Lperhour*ft3perL;  # volume breathed per hour in ft^3, approximately 16 ft^3

Nhour = 6;        # of hours in classroom per day
Nday = 5;         # number of days in school
t = Nhour*Nday;   # total number of hours exposed

Nkid = 20;        # of kids in classroom
Nweek = 15;       # of weeks of school

# calculate nroom = number of virus particles per cubic foot in room air
def calc_nroom(Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
               Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m):
    # to avoid divide by zero, we can't allow the denominator Rhvac + Rfan + Rleak + Nhepa*Rhepa=0 and Rhvac=0
    if Rfan + Rleak + Nhepa*Rhepa==0:
        if isscalar(Rhvac):
            if Rhvac==0: Rhvac=1
        else: Rhvac[Rhvac==0]=1
    return ( (1-m/100)*p*Nsick + nhall*(Rleak+(1-Ffresh/100)*Rhvac) ) / ( Rhvac + Rfan + Rleak + Nhepa*Rhepa*Fhepa/100 )

# 1-week probability of your kid getting sick
def calc_prob(t=t,Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
              Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m,k=k,Vbreath=Vbreath,nroom=None):
    if nroom is None:
        nroom=calc_nroom(Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
                         Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m)
    return 1-exp(-(1-m/100)*Vbreath*t*nroom/k)

In [3]:
# nbi:hide_in

# left plot: compares mask & no-mask (assuming 1 HEPA filter)
# right plot: compares # of HEPA filters (assuming mask)
# assumes that Ffresh = 100% (or equivalently, assumes that nhall=0)
def plot_risk1(Nhepa, Rhepa, Fhepa, m, p, k, Nsick, t):

    fig, ax = subplots(1,2,figsize=(10,4))

    Rfresh_arr = linspace(0,1000,51)

    nroom_arr_mask = calc_nroom(Rhvac=Rfresh_arr,Ffresh=100,Rfan=0,Rleak=0,nhall=0,\
                                      Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m)
    nroom_arr_no_mask = calc_nroom(Rhvac=Rfresh_arr,Ffresh=Ffresh,Rfan=0,Rleak=Rleak,nhall=nhall,\
                                         Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=0)

    prob_arr_mask = calc_prob(m=m,k=k,Vbreath=16,t=t,nroom=nroom_arr_mask)
    prob_arr_no_mask = calc_prob(m=0,k=k,Vbreath=16,t=t,nroom=nroom_arr_no_mask)
    
    ax[0].plot(Rfresh_arr,100*prob_arr_mask,'b-',label='mask')
    ax[0].plot(Rfresh_arr,100*prob_arr_no_mask,'r-',label='no mask')
    ax[0].set_xlabel('Rfresh (ft$^3$/min)',fontsize=12)
    ax[0].set_ylabel('probability (%)',fontsize=12)
    ax[0].set_xlim([0,1000])
    ymax=(ax[0].get_ylim())[1]
    ax[0].set_ylim([0,ymax])
    ax[0].legend(fontsize=12)
    ax[0].set_title('Covid-19 transmission\n({:,g} HEPA filter{}/classroom)'.format(Nhepa,'' if Nhepa==1 else 's'))
    
    carr = 'rbgmkcy'
    for N in arange(4):
        nroom_arr_mask = calc_nroom(Rhvac=Rfresh_arr,Ffresh=100,Rfan=0,Rleak=0,nhall=0,\
                                      Nhepa=N,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m)
        prob_arr_mask = calc_prob(m=m,k=k,Vbreath=16,t=t,nroom=nroom_arr_mask)
        ax[1].plot(Rfresh_arr,100*prob_arr_mask,color=carr[N],ls='-',label='{:,g} HEPA'.format(N))
    ax[1].set_xlabel('Rfresh (ft$^3$/min)',fontsize=12)
    ax[1].set_ylabel('probability (%)',fontsize=12)
    ax[1].set_xlim([0,1000])
    y1 = (ax[1].get_ylim())[1]
    y2 = 2*100*calc_prob(Rhvac=100,Ffresh=100,Rfan=0,Rleak=0,nhall=0,Nhepa=0,Rhepa=Rhepa,Fhepa=Fhepa,\
                         Nsick=Nsick,p=p,m=m,k=k,Vbreath=16,t=t)
    ymax = min(y1,y2)
    ax[1].set_ylim([0,ymax])
    ax[1].legend(fontsize=12)
    ax[1].set_title('Covid-19 transmission\n({:,g}% mask filtration effectiveness)'.format(m))

    fig.tight_layout()

In [4]:
# nbi:hide_in
def reset_slider1_defaults(*args):
    Nhepa=1; Rhepa=240; Fhepa=99.9; m=25; p=33; k=410; Nsick=1; t=30;
    
    Nhepa_slider1.value = Nhepa
    Rhepa_slider1.value = Rhepa
    Fhepa_slider1.value = Fhepa
    m_slider1.value = m
    p_slider1.value = p
    k_slider1.value = k
    Nsick_slider1.value = Nsick
    t_slider1.value = t
    
# Some general layouts for the sliders
layout_cen = widgets.Layout(width='100%', display='flex', justify_content='center', align_items='center')
layout_right = widgets.Layout(width='100%', display='flex', justify_content='flex-end', align_items='flex-end')
layout_button = widgets.Layout(width='80%', border = '2px solid black')
layout_wid = widgets.Layout(width='250px')

style_cen = {'justify-content': 'center'}
style_narrow = {'description_width': '60px'}
style_med = {'description_width': '60px'}

w1 = widgets.interactive(plot_risk1, \
                        Nhepa=widgets.IntSlider(min=0,max=5,step=1,value=0,description=r'$N_{\text{hepa}}$',\
                                                  continuous_update=False,style=style_med,layout=layout_wid), \
                        Rhepa=widgets.IntSlider(min=0,max=1000,step=10,value=Rhepa,description=r'$R_{\text{hepa}}$',\
                                                  continuous_update=False,style=style_med,layout=layout_wid), \
                        Fhepa=widgets.FloatSlider(min=70,max=100,step=0.01,value=Fhepa,description=r'$F_{\text{hepa}}$',\
                                                  continuous_update=False,style=style_med,layout=layout_wid), \
                        m=widgets.IntSlider(min=0,max=100,step=1,value=m,description=r'$F_{\mathrm{mask}}$',\
                                                  continuous_update=False,style=style_med,layout=layout_wid), \
                        p=widgets.IntSlider(min=0,max=300,step=10,value=p,description=r'$p$ (shed)',\
                                                  continuous_update=False,style=style_med,layout=layout_wid), \
                        k=widgets.IntSlider(min=10,max=1000,step=10,value=k,description=r'$k$ (dose)',\
                                                  continuous_update=False,style=style_med,layout=layout_wid), \
                        Nsick=widgets.IntSlider(min=0,max=5,step=1,value=Nsick,description=r'$N_{\text{sick}}$',\
                                                  continuous_update=False,style=style_med,layout=layout_wid), \
                        t=widgets.IntSlider(min=0,max=40,step=1,value=t,description=r'$t$ (hrs)',\
                                                  continuous_update=False,style=style_med,layout=layout_wid))

Nhepa_slider1 = w1.children[0]
Rhepa_slider1 = w1.children[1]
Fhepa_slider1 = w1.children[2]
m_slider1 = w1.children[3]
p_slider1 = w1.children[4]
k_slider1 = w1.children[5]
Nsick_slider1 = w1.children[6]
t_slider1 = w1.children[7]
out1 = w1.children[-1]

hepa_label1 = widgets.VBox([widgets.HTML(value=f"<b>HEPA controls</b>")],layout=layout_cen)
virus_label1 = widgets.VBox([widgets.HTML(value=f"<b>Virus controls</b>")],layout=layout_cen)
expose_label1 = widgets.VBox([widgets.HTML(value=f"<b>Exposure controls</b>")],layout=layout_cen)
reset_button1 = widgets.Button(description='Reset all controls',layout=layout_button)
reset_button1.on_click(reset_slider1_defaults)
reset_box1 = widgets.VBox([reset_button1],layout=layout_cen)
v1a = widgets.VBox([hepa_label1,Nhepa_slider1,Rhepa_slider1,Fhepa_slider1])
v1b = widgets.VBox([virus_label1,m_slider1,p_slider1,k_slider1])
v1c = widgets.VBox([expose_label1,Nsick_slider1,t_slider1,reset_box1])
h1 = widgets.HBox([v1a,v1b,v1c])
vtot1 = widgets.VBox([out1,h1])

display(vtot1)
reset_slider1_defaults()

### Some Comments


- What is a typical value for the fresh air flow rate? Modern building code requires at least 4 full air turnovers per hour, so in a classroom of 20 ft $\times$ 30 ft (with 9 ft ceilings), the total air flow should be at least 360 ft$^3$/min. However, older buildings may not have been subject to the same building code, and/or their HVAC systems may have deteriorated over time. In fact, there's an even more important consideration: in climates requiring indoor air temperature control, the HVAC system may recirculate up to 90% of the air! Building code requires only 10% of the incoming air to be fresh outdoor air, so a typical fresh air flow rate on a cold winter day may be only 36 ft$^3$/min.


- Now there's an even bigger problem: what if a child is sick in another classroom, and 90% of *their* air is circulated into *your* classroom? The best case scenario would completely avoid mixing air between classrooms, but real school buildings are rarely so simple. To avoid transmission between classrooms, in some circumstances where $F_{\mathrm{fresh}}$ is low it may be best to turn down the total HVAC flow rate as low as possible (even if that means reducing the amount of fresh air), and instead use more HEPA filters to reduce contamination within each classroom individually. We explore these circumstances in more detail [below](#equations) by adding a concentration $n_{\mathrm{hall}}$ of virus particles in the building, but outside the classroom. (The simulations above use the same equations but with $n_{\mathrm{hall}}=0$.)


- The plots are drawn from the point of view of a parent: they show the probability $p$ that *your* child will get sick. But from the school's point of view, in a classroom of $N$ students with $N_{\mathrm{sick}} > 0$, the total probabiliy that *somebody's* child (or teacher) will get sick is $P_{\mathrm{tot}} = 1-(1-p)^N$. So even for a relatively small *individual* transmission risk of $p=2\%$, the *collective* probability of transmission within a single classroom of $N=20$ students is 33%.


- A few words on the other parameters (more detail with references [below](#virus_parameters)).

  - $F_{\mathrm{mask}}$ is the % effectiveness of a simple cloth face covering. It's never going to be 100%, and studies suggest it's probably more like 25%. Of course if you're not wearing a mask, then $F_{\mathrm{mask}}$ will be 0%.
  
  - $t$ is the number of hours of exposure to a sick individual. Adults are typically shedding SARS-CoV-2 virus for 1 to 6 days before they are symptomatic. Young children may never show symptoms, so they may shed virus for at least this long, until an older member of their family becomes symptomatic, tests positive, and triggers the quarantine of the infected child.
  
  - $p$ is the number of virus particles shed by a sick individual per minute; $k$ is the total number of virus particles required to trigger an infection. These numbers are not yet well-known for SARS-CoV-2, but I've used some preliminary estimates from the scientific literature, based on previous studies of influenza and the first SARS epidemic in 2003.
  
  
- Why move the teachers instead of the students? With 4 air turnovers per hour, it will take at least 15 minutes to decontaminate a classroom. SARS-COV-2 virus remained viable in aerosol throughout the duration of a 3-hour experiment, with a half-life of 1.2 hours. Surfaces are not the primary cause of transmission, but they may become contaminated by aerosols even if they are not directly touched by a sick individual. On surfaces, the half-life is $\sim$3 hours on cardboard, $\sim$5 hours on steel, and $\sim$7 hours on plastic, although some viable SARS-COV-2 virus could be detected on all surfaces for at least 24 hours (<a href="http://dx.doi.org/10.1056/nejmc2004973" target="_blank">van Doremalen 2020</a>).


- Why conduct classes outside? A Chinese preprint studied 318 outbreaks of size three or more, involving 1245 total confirmed cases in 120 cities. Out of all 318 outbreaks, only a single one involved outdoor transmission (<a href="https://doi.org/10.1101/2020.04.04.20053058" target="_blank">Qian 2020</a>).

  
- A nerd's note on terminology: SARS-CoV-2 is the name of the virus (i.e. the particle itself). Covid-19 is the name of the disease (i.e. the set of symptoms) triggered by the virus.

### More Details

<a id="equations"></a>
**Equations**

The equilibrium concentration of virus particles in a classroom is given by the steady state rate equation (particles coming in = particles going out):

\begin{equation}
\underbrace{(1-F_{\mathrm{mask}})\,p\,N_{\mathrm{sick}}}_{\substack{\text{particles/minute} \\ \text{added by sick} \\ \text{people in room} }}
= \underbrace{(F_{\mathrm{fresh}} R_{\mathrm{hvac}} + R_{\mathrm{fan}}) n_{\mathrm{room}}}_{\substack{\text{particles/minute} \\ \text{replaced by fresh air} }}
+\underbrace{[(1-F_{\mathrm{fresh}}) R_{\mathrm{hvac}} + R_{\mathrm{leak}}] (n_{\mathrm{room}}-n_{\mathrm{hall}})}_{\substack{\text{particles/minute} \\ \text{replaced by hall air} }}
+\underbrace{N_{\mathrm{hepa}} R_{\mathrm{hepa}} F_{\mathrm{hepa}} n_{\mathrm{room}}}_{\substack{\text{particles/minute} \\ \text{removed by HEPA} }}
\end{equation}

Solving for $n_{\mathrm{room}}$, we find:

\begin{equation}
n_{\mathrm{room}} = \frac{ (1-F_{\mathrm{mask}}) \, p \, N_{\mathrm{sick}} + n_{\mathrm{hall}} \left[ R_{\mathrm{leak}} + (1-F_{\mathrm{fresh}}) R_{\mathrm{hvac}} \right] }
{ R_{\mathrm{hvac}} + R_{\mathrm{fan}} + R_{\mathrm{leak}} + N_{\mathrm{hepa}} R_{\mathrm{hepa}} F_{\mathrm{hepa}} }
\end{equation}

Now that we know the equilibrium concentration of virus particles per cubic foot in the room $n_{\mathrm{room}}$, the probability that you will get sick increases with the amount of time $t$ you spend in the room:

\begin{equation}
p(t) = 1 - \exp[-(1-F_{\mathrm{mask}}) \, V_{\mathrm{breath}} \, t \, n_{\mathrm{room}} \, / \, k]
\end{equation}

These full equations are simulated below.

In [6]:
# nbi:hide_in
def reset_slider_defaults(*args):
    Rhvac=360; Ffresh=10; Rfan=0; Rleak=0; nhall=0; Nhepa=1; Rhepa=240; Fhepa=99.9;
    Nsick=1; p=33; m=25; k=410; Vbreath=16; t=30;
    
    Rhvac_slider.value = Rhvac
    Ffresh_slider.value = Ffresh
    Rfan_slider.value = Rfan
    Rleak_slider.value = Rleak
    nhall_slider.value = nhall
    Nhepa_slider.value = Nhepa
    Rhepa_slider.value = Rhepa
    Fhepa_slider.value = Fhepa
    Nsick_slider.value = Nsick
    p_slider.value = p
    m_slider.value = m
    k_slider.value = k
    Vbreath_slider.value = Vbreath
    t_slider.value = t
    
def plot_risk(Rhvac, Ffresh, Rfan, Rleak, nhall, Nhepa, Rhepa, Fhepa, Nsick, p, m, k, Vbreath, t):
    
    fig, ax = subplots(1,3,figsize=(14,4))

    nhall_threshold = calc_nroom(Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
                                 Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m)

    Rhvac_arr = linspace(0,1000,51)
    nroom_arr_mask_Rhvac = calc_nroom(Rhvac=Rhvac_arr,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
                                      Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m)
    nroom_arr_no_mask_Rhvac = calc_nroom(Rhvac=Rhvac_arr,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
                                         Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=0)

    Ffresh_arr = linspace(0,100,51)
    nroom_arr_mask_Ffresh = calc_nroom(Rhvac=Rhvac,Ffresh=Ffresh_arr,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
                                       Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m)
    nroom_arr_no_mask_Ffresh = calc_nroom(Rhvac=Rhvac,Ffresh=Ffresh_arr,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
                                          Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=0)

    # if the concentration in the hall is low, then Ffresh matters little, so we fix Ffresh and vary Rhvac
    if nhall < nhall_threshold:       
        ax[0].plot(Rhvac_arr,nroom_arr_mask_Rhvac,'b-',label='mask')
        ax[0].plot(Rhvac_arr,nroom_arr_no_mask_Rhvac,'r-',label='no mask')
        ax[0].set_xlabel('Rhvac (ft$^3$/min)',fontsize=12)
        ax[0].set_ylabel('particles per ft$^3$',fontsize=12)
        ax[0].set_xlim([0,1000])
        ymax=(ax[0].get_ylim())[1]
        ax[0].set_ylim([0,ymax])
        ax[0].vlines(Rhvac, 0, ymax, colors='gray', linestyles='dashed', label='Rhvac')
        ax[0].legend(fontsize=12)
        ax[0].set_title('Virus concentration in classroom')
        
    # if the concentration in the hall is high, then Ffresh matters a lot, so we fix Rhvac and vary Ffresh
    else:       
        ax[0].plot(Ffresh_arr,nroom_arr_mask_Ffresh,'b-',label='mask')
        ax[0].plot(Ffresh_arr,nroom_arr_no_mask_Ffresh,'r-',label='no mask')
        ax[0].set_xlabel('fresh air (%)',fontsize=12)
        ax[0].set_ylabel('particles per ft$^3$',fontsize=12)
        ax[0].set_xlim([0,100])
        ymax=(ax[0].get_ylim())[1]
        ax[0].set_ylim([0,ymax])
        ax[0].vlines(Ffresh, 0, ymax, colors='gray', linestyles='dashed', label='Ffresh')
        ax[0].legend(fontsize=12)
        ax[0].set_title('Virus concentration in classroom')

    prob_arr_mask_Rhvac = calc_prob(m=m,k=k,Vbreath=Vbreath,t=t,nroom=nroom_arr_mask_Rhvac)
    prob_arr_no_mask_Rhvac = calc_prob(m=0,k=k,Vbreath=Vbreath,t=t,nroom=nroom_arr_no_mask_Rhvac)

    ax[1].plot(Rhvac_arr,100*prob_arr_mask_Rhvac,'b-',label='mask')
    ax[1].plot(Rhvac_arr,100*prob_arr_no_mask_Rhvac,'r-',label='no mask')
    ax[1].set_xlabel('Rhvac (ft$^3$/min)',fontsize=12)
    ax[1].set_ylabel('probability (%)',fontsize=12)
    ax[1].set_xlim([0,1000])
    ymax=(ax[1].get_ylim())[1]
    ax[1].set_ylim([0,ymax])
    ax[1].vlines(Rhvac, 0, ymax, colors='gray', linestyles='dashed', label='Rhvac')
    ax[1].legend(fontsize=12)
    ax[1].set_title('Covid-19 transmission\n(fixed Ffresh={:,g}%)'.format(Ffresh))
    
    prob_arr_mask_Ffresh = calc_prob(m=m,k=k,Vbreath=Vbreath,t=t,nroom=nroom_arr_mask_Ffresh)
    prob_arr_no_mask_Ffresh = calc_prob(m=0,k=k,Vbreath=Vbreath,t=t,nroom=nroom_arr_no_mask_Ffresh)

    ax[2].plot(Ffresh_arr,100*prob_arr_mask_Ffresh,'b-',label='mask')
    ax[2].plot(Ffresh_arr,100*prob_arr_no_mask_Ffresh,'r-',label='no mask')
    ax[2].set_xlabel('fresh air (%)',fontsize=12)
    ax[2].set_ylabel('probability (%)',fontsize=12)
    ax[2].set_xlim([0,100])
    ymax=(ax[2].get_ylim())[1]
    ax[2].set_ylim([0,ymax])
    ax[2].vlines(Ffresh, 0, ymax, colors='gray', linestyles='dashed', label='Ffresh')
    ax[2].legend(fontsize=12)
    ax[2].set_title('Covid-19 transmission\n(fixed Rhvac={:,g} ft$^3$/min)'.format(Rhvac))
    
    fig.tight_layout()

In [7]:
# nbi:hide_in

w = widgets.interactive(plot_risk, \
                        Rhvac=widgets.IntSlider(min=0,max=1000,step=10,value=Rhvac,description=r'$R_{\text{hvac}}$',\
                                                continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Ffresh=widgets.IntSlider(min=0,max=100,step=1,value=Ffresh,description=r'$F_{\text{fresh}}$',\
                                                continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Rfan=widgets.IntSlider(min=0,max=2500,step=10,value=Rfan,description=r'$R_{\text{fan}}$',\
                                                continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Rleak=widgets.IntSlider(min=0,max=200,step=10,value=Rleak,description=r'$R_{\text{leak}}$',\
                                                continuous_update=False,style=style_narrow,layout=layout_wid), \
                        nhall=widgets.FloatSlider(min=0,max=1,step=0.01,value=nhall,description=r'$n_{\text{hall}}$',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Nhepa=widgets.IntSlider(min=0,max=5,step=1,value=Nhepa,description=r'$N_{\text{hepa}}$',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Rhepa=widgets.IntSlider(min=0,max=1000,step=10,value=Rhepa,description=r'$R_{\text{hepa}}$',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Fhepa=widgets.FloatSlider(min=70,max=100,step=0.01,value=Fhepa,description=r'$F_{\text{hepa}}$',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Nsick=widgets.IntSlider(min=0,max=5,step=1,value=Nsick,description=r'$N_{\text{sick}}$',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        p=widgets.IntSlider(min=0,max=300,step=10,value=p,description=r'$p$ (shed)',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        m=widgets.IntSlider(min=0,max=100,step=1,value=m,description=r'$F_{\mathrm{mask}}$',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        k=widgets.IntSlider(min=10,max=1000,step=10,value=k,description=r'$k$ (dose)',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        Vbreath=widgets.IntSlider(min=10,max=20,step=1,value=Vbreath,description=r'$V_{\text{breath}}$',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid), \
                        t=widgets.IntSlider(min=0,max=40,step=1,value=t,description=r'$t$ (hrs)',\
                                                  continuous_update=False,style=style_narrow,layout=layout_wid))

Rhvac_slider = w.children[0]
Ffresh_slider = w.children[1]
Rfan_slider = w.children[2]
Rleak_slider = w.children[3]
nhall_slider = w.children[4]
Nhepa_slider = w.children[5]
Rhepa_slider = w.children[6]
Fhepa_slider = w.children[7]
Nsick_slider = w.children[8]
p_slider = w.children[9]
m_slider = w.children[10]
k_slider = w.children[11]
Vbreath_slider = w.children[12]
t_slider = w.children[13]
out = w.children[-1]

hvac_label = widgets.VBox([widgets.HTML(value=f"<b>HVAC controls</b>")],layout=layout_cen)
hepa_label = widgets.VBox([widgets.HTML(value=f"<b>HEPA controls</b>")],layout=layout_cen)
virus_label = widgets.VBox([widgets.HTML(value=f"<b>Virus controls</b>")],layout=layout_cen)
expose_label = widgets.VBox([widgets.HTML(value=f"<b>Exposure controls</b>")],layout=layout_cen)
reset_button = widgets.Button(description='Reset all controls',layout=layout_button)
reset_button.on_click(reset_slider_defaults)
reset_box = widgets.VBox([reset_button],layout=layout_cen)
va = widgets.VBox([hvac_label,Rhvac_slider,Ffresh_slider,Rfan_slider,Rleak_slider,nhall_slider])
vb = widgets.VBox([hepa_label,Nhepa_slider,Rhepa_slider,Fhepa_slider])
vc = widgets.VBox([virus_label,m_slider,p_slider,k_slider])
vd = widgets.VBox([expose_label,Nsick_slider,Vbreath_slider,t_slider,reset_box])
h = widgets.HBox([va,vb,vc,vd])
vtot = widgets.VBox([out,h])

display(vtot)
reset_slider_defaults()

### Some Comments

* This simulation starts by assuming there is only one sick child in *your* classroom ($N_{\mathrm{sick}}=1$) and there are no other sick children in the school ($n_{\mathrm{hall}}=0$). In this case, we see no dependence on the fresh air fraction $F_{\mathrm{fresh}}$ (right plot), as expected, because air from the hallway is just as good as outside air.


* From this starting point, we see that a typical virus concentration in a room with one sick child is on order 0.1 to 1 particles/ft$^3$. So we can start to increase $n_{\mathrm{hall}}$ towards this range of values, and now we see that the risk decreases with increasing fresh air fraction $F_{\mathrm{fresh}}$.


* If we increase $n_{\mathrm{hall}}$ above $n_{\mathrm{room}}$ (i.e. if there are more sick kids outside of your classroom than inside your classroom), then we see the danger of recirculated airflow. Unless $F_{\mathrm{fresh}}$ is close to 100%, it's better to lower the total HVAC airflow ($R_{\mathrm{HVAC}}$) to avoid bringing virus into your classroom. 


* If the HVAC system cannot be configured to provide 4 full turnovers of *fresh* air per hour, another option in moderate climates is to use box fans in open windows. Box fans in windows can provide 1000-2000 ft$^3$/min of fresh air (assuming the exhaust impedance is low enough).

In [11]:
# nbi:hide_in
Nstudent = 20;    # of students in classroom
Nweek = 15;       # of weeks of school
prob1 = 0.01;     # probability that 1 student shows up sick in any given week

# cumulative probability of your kid getting sick over Nweek weeks
def calc_prob_weeks(Nweek=Nweek,t=t,Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
              Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m,k=k,Vbreath=Vbreath,nroom=None,prob=None):
    if prob is None:
        prob=calc_prob(t=t,Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
              Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m,k=k,Vbreath=Vbreath,nroom=nroom)
    return 1-(1-prob)**Nweek

# cumulative probability of one transmission event in a classroom over one week
def calc_prob_class(Nstudent=Nstudent,t=t,Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
              Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m,k=k,Vbreath=Vbreath,nroom=None,prob=None):
    if prob is None:
        prob=calc_prob(t=t,Rhvac=Rhvac,Ffresh=Ffresh,Rfan=Rfan,Rleak=Rleak,nhall=nhall,\
              Nhepa=Nhepa,Rhepa=Rhepa,Fhepa=Fhepa,Nsick=Nsick,p=p,m=m,k=k,Vbreath=Vbreath,nroom=nroom)
    return 1-(1-prob)**Nstudent


### Parameters

<a id="hvac_parameters"></a>
**HVAC controls**

Standard building code requires that each room has at least 4 full air exchanges per hour. A typical classroom size is 20 ft $\times$ 30 ft, with height 9 ft, giving a total volume of 5400 ft$^3$. So the HVAC system must push 4 $\times$ 20 ft $\times$ 30 ft $\times$ 9 ft per 60 min, i.e. $\boxed{R_{\mathrm{HVAC}} = 360\; \text{ft}^3/\text{min}}$. But standard building code allows up to 90% of the air exchange to recirculate within the building (i.e. as little as 10\% may be fresh air). In a cold climate in the winter, the indoor temperature must remain above 64$^{\circ}$F, so the fraction of fresh air may be very close to the minimum allowed, $\boxed{F_{\mathrm{fresh}} = 10\%}$. It's also worth noting that building code may have been less stringent when old school buildings were constructed. Furthermore, aging HVAC systtems that used to deliver 4 full air exchanges per hour may no longer do so, due to declining pump efficiency or residue buildup that impedes airflow in the ductwork.

The recirculated air $(1-F_{\mathrm{fresh}})R_{\mathrm{HVAC}}$ may contain a concentration of virus particles shed by sick occupants from elsewhere in the building. We call this $n_{\mathrm{hall}}$, measured in particles per ft$^3$.  For this simulation, we start with the best case assumption that there is only a single sick person in the whole building, and we consider the contagion within the classroom of that one sick person. In that case $\boxed{n_{\mathrm{hall}}=0\,/\text{ft}^3}$. We note that a more conservative assumption is that the virus concentration in the hallway may be similar to that in the classroom if there are multiple sick children moving through the building between classes, i.e. $n_{\mathrm{hall}}$ could be on order 0.1 particle per ft$^3$ or higher.

In moderate climates, we may input additional fresh air using window box fans, but this is impractical in the winter for much of the country, so we start with $\boxed{R_{\mathrm{fan}}=0\; \text{ft}^3/\text{min}}$.

It's almost impossible to perfectly balance the HVAC system such that the air intake exactly equals the exhaust for a given room. In practice, there is usually some leakage of air from the hallway into the room (in addition to the $(1-F_{\mathrm{fresh}})R_{\mathrm{HVAC}}$ that is intentionally recirculated into the room). A typical value of $R_{\mathrm{leak}}$ might be 50 ft$^3$/min, but we start with the conservative assumption that $\boxed{R_{\mathrm{leak}} = 0\;\text{ft}^3/\text{min}}$.

**HEPA controls**

A free-standing HEPA filter can help significantly to reduce the viral concentration in the classroom air. For example, we start by considering a medical-grade [HEPA filter](https://www.amazon.com/gp/product/B08194ZQ4N/) that removes $\boxed{F_{\mathrm{HEPA}} = 99.9\%}$ of all particles greater than 0.1 $\mu$m. This product claims to clear 1,600 ft$^2$ per hour. Assuming a ceiling height of 9 ft, we calculate 1600 ft$^2$ $\times$ 9 ft / 60 min, yeilding $\boxed{R_{\mathrm{HEPA}} = 240\; \text{ft}^3/\text{min}}$. This particular product costs \\$250, and requires a replacement filter for \$65 every 6 months.

<a id="virus_parameters"></a>
**Virus shedding**

The rate of virus particles $p$ shed by a sick person is not yet well-known for COVID-19, but we take some estimates from Prof. Erin Bromage's [blog post](https://www.erinbromage.com/post/the-risks-know-them-avoid-them) and the peer-reviewed scientific references within. One study showed that college students with influenza can shed $\sim 30$ infectious viral particles per minute [(Yan 2018)](http://dx.doi.org/10.1073/pnas.1716561115). Another early study of adults with COVID-19 showed that quantitative SARS-CoV-2 viral loads were similarly high in four symptom groups (adults with typical symptoms, those with atypical symptoms, those who were presymptomatic, and those who remained asymptomatic). It is notable that 17 of 24 specimens (71%) from presymptomatic persons had viable SARS-CoV-2 virus by culture 1 to 6 days before the development of symptoms [(Gandhi 2020)](http://dx.doi.org/10.1056/NEJMe2009758). We start with a conservative estimate of SARS-COV-2 viral shedding as $\boxed{p=33\,/\text{min}}$.

**Infectious dose**

The infectious dose $k$ is the number of virus particles required to infect a healthy person. For MERS, $k$ is somewhere in the thousands, perhaps as high as 10,000. Prof. Willem van Schaik (U. Birmingham) estimates that for COVID-19, the infectious dose in the high hundreds or low thousands. It’s reasonable to assume it takes fewer particles to launch an infection in the case of COVID-19 than MERS, because COVID-19 has shown itself to be much more transmissible. Each person with Covid-19 infects two or three others on average, while for MERS that number is less than one [(Vox)](https://www.vox.com/future-perfect/2020/4/24/21233226/coronavirus-runners-cyclists-airborne-infectious-dose).

The infectious dose of influenza was shown to be a few 1000s of virus particles [(Nitikin 2014)](https://doi.org/10.1155/2014/859090). The infectious dose of SARS-COV-1 virus was found to be 410 PFU/mL [(Watanabe 2010)](http://dx.doi.org/10.1111/j.1539-6924.2010.01427.x).

<!-- The units used by experimental biologists to quantify the number of virus particles are confusing! Basically, biologists spread a diluted a virus solution on a petri dish and count colonies a few days later. They report the results in one of two standard units: PFU/mL (plaque-forming-units per milliLiter) or TCID$_{50}$ (50% Tissue Culture Infective Dose), which are roughly equivalent. To be a little more precise, 0.69 PFU/mL = 1 TCID$_{50}$ [(Wikipedia)](https://en.wikipedia.org/wiki/Virus_quantification). But how does this translate to actual virus particles? Unfortunately, the translation is not universal, because it has to do with how many virus particles are actually viable to infect (since viruses replicate single-stranded RNA rather than double-stranded DNA, they are more susceptible to mutations that make them non-viable). One set of studies says the ratio of total particles to PFU for influenza and other RNA viruses is on the order of 10:1 to 100:1 [(Fonville 2015)](http://dx.doi.org/10.1371/journal.ppat.1005204). Another set of studies says that 300–650 copies of human influenza viruses were contained in 1 TCID$_{50}$ [(Nitikin 2014)](https://doi.org/10.1155/2014/859090).
--> 

Putting this all together, we start with an estimate of the infectious dose as $\boxed{k=410}$ virus particles.

**Masks**

One early study of 4 adults with COVID-19 showed that wearing a surgical mask completely eliminated detectable aerosolized SARS-CoV-2 virus particles over a 30 minute collection period [(Leung 2020)](http://dx.doi.org/10.1038/s41591-020-0843-2). But that is a pretty small study, and it's hard to draw reliable conclusions. A larger study of 37 patients with influenza showed that wearing a sugical mask led to a 2.8$\times$ reduction in detectable aerosolized influenza virus particles [(Milton 2013)](http://dx.doi.org/10.1371/journal.ppat.1003205). A more complete review (see <a href="https://static1.squarespace.com/static/5e8126f89327941b9453eeef/t/5f2c4463a5c9f75a38d2b26f/1596736614213/N95DECON_cloth_mask_breathability_filtrationtechnical_report_v1_200804.pdf" target="_blank">Fig. 3 on page 27</a>) shows that woven or knit cloth is more porous than the microfiber material of a surgical mask, so a simple cloth mask may be only $\sim$25% or less efficient. We start our simulation with the estimate that $\boxed{F_{\mathrm{mask}}=25\%}$ of virus particles are filtered by a properly-worn cloth mask (note that $F_{\mathrm{mask}}=100\%$ if you have an impenetrable mask, while $F_{\mathrm{mask}}=0\%$ if you're not wearing a mask).

**Exposure controls**

A typical adult breaths $\boxed{V_{\mathrm{breath}} \approx 16\;\text{ft}^3/\text{hour}}$ (a child will be somewhat less).

The exposure time is $\boxed{t=30\;\text{hrs}}$ for a school week of five 6-hour days. Adults are typically infectious for 1 to 6 days before symptoms, while children may be even less likely to develop warning symptoms while infectious. It seems likely that if a child becomes infected, they may be inadvertently shedding virus in the classroom for a full week before they either recover, or a family member becomes ill and the child is sent home to quarantine. 

To estimate $N_{\mathrm{sick}}$ for your geographical region, here is a [risk assessment tool](https://covid19risk.biosci.gatech.edu/) showing the likelihood that at least one person will be sick in a gathering of a given size. For starters, we use $\boxed{N_{\mathrm{sick}} = 1}$, and you can just scale the plots by the appropriate factor for your geographical region.

#### Disclaimer

I am not a medical doctor or a public health expert. I am just a <a href="https://www.physics.harvard.edu/people/facpages/hoffman">physicist</a> and a mom who is trying to figure out whether it's safe to send my kids back to school. I have read a modest number of peer-reviewed articles in reputable medical journals, but I have by no means reviewed the full body of rapidly-evolving scientific literature on the topic of COVID-19.

#### Acknowledgments

Thanks to John Doyle and Larissa Little and the <a href="https://www.n95decon.org/">N95decon</a> team for compiling many of the equations and references. Thanks to Ruizhe Kang and Harry Pirie for advice about Python.

#### References

Yan *et al*, "Infectious virus in exhaled breath of symptomatic seasonal influenza cases from a college community", Proc. National Academy of Sciences (2018) [10.1073/pnas.1716561115](https://dx.doi.org/10.1073/pnas.1716561115)

Gandhi *et al*, "Asymptomatic Transmission, the Achilles’ Heel of Current Strategies to Control Covid-19", New England Journal of Medicine (2020)
[10.1056/NEJMe2009758](http://dx.doi.org/10.1056/NEJMe2009758)

Leung *et al*, "Respiratory virus shedding in exhaled breath and efficacy of face masks", Nature Medicine (2020) [10.1038/s41591-020-0843-2](http://dx.doi.org/10.1038/s41591-020-0843-2)

Milton *et al*, "Influenza Virus Aerosols in Human Exhaled Breath: Particle Size, Culturability, and Effect of Surgical Masks", PLOS Pathogens (2013) [10.1371/journal.ppat.1003205](http://dx.doi.org/10.1371/journal.ppat.1003205)

van Doremalen *et al*, "Aerosol and Surface Stability of SARS-CoV-2 as Compared with SARS-CoV-1", New England Journal of Medicine (2020) [10.1056/nejmc2004973](http://dx.doi.org/10.1056/nejmc2004973)

Qian *et al*, "Indoor transmission of SARS-CoV-2", MedRxiv preprint (2020) [10.1101/2020.04.04.20053058](http://dx.doi.org/10.1101/2020.04.04.20053058)

Alford *et al*, "Human Influenza Resulting from Aerosol Inhalation", Proc. Soc. Experimental Biology & Medicine (1966) [10.3181/00379727-122-31255](http://dx.doi.org/10.3181/00379727-122-31255)

Nikitin *et al*, "Influenza Virus Aerosols in the Air and Their Infectiousness", Advances in Virology (2014) [10.1155/2014/859090](http://dx.doi.org/10.1155/2014/859090)

Watanabe *et al*, "Development of a Dose-Response Model for SARS Coronavirus", Risk Analysis (2010) [10.1111/j.1539-6924.2010.01427.x](http://dx.doi.org/10.1111/j.1539-6924.2010.01427.x)

Fonville *et al*, "Influenza Virus Reassortment Is Enhanced by Semi-infectious Particles but Can Be Suppressed by Defective Interfering Particles", PLOS Pathogens (2015), [10.1371/journal.ppat.1005204](http://dx.doi.org/10.1371/journal.ppat.1005204)

"Virus Quantification", [Wikipedia](https://en.wikipedia.org/wiki/Virus_quantification)

#### Feedback

Feedback welcome <a href="https://www.facebook.com/permalink.php?story_fbid=1218601725147767&id=100009938524181" target="_blank">here</a>.